<a href="https://colab.research.google.com/github/KatrinaH92/git_test/blob/main/Liberal_conservative_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import numpy as np
import tensorflow as tf
import json
import scipy

import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets, metrics, Sequential, utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

from tensorflow.keras.layers import Dense, Embedding, LSTM
import pandas as pd
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [39]:
BATCH_SIZE = 512

In [40]:
METRICS = [
      lambda : tf.keras.metrics.TruePositives(name='tp'),
      lambda : tf.keras.metrics.FalsePositives(name='fp'),
      lambda : tf.keras.metrics.TrueNegatives(name='tn'),
      lambda : tf.keras.metrics.FalseNegatives(name='fn'),

      lambda : tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      lambda : tf.keras.metrics.Precision(name='precision'),
      lambda : tf.keras.metrics.Recall(name='recall'),
]

def fresh_metrics():
    return [metric() for metric in METRICS]

In [41]:
features = data[['Title', 'Text']]  # Use these as input features
labels = data['Political Lean']  # Target variable
features.fillna("", inplace=True)
#Combine 'Title' and 'Text' into a single feature
features['combined_text'] = features['Title'] + " " + features['Text']

# Tokenize the text
tokenizer = Tokenizer(num_words=10000)  # Limit to the top 10,000 words
tokenizer.fit_on_texts(features['combined_text'])
sequences = tokenizer.texts_to_sequences(features['combined_text'])

# Pad sequences to ensure uniform length
padded_sequences = pad_sequences(sequences, maxlen=100)  # Limit sequence length to 100

# Encode target labels as integers
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# If required, convert to one-hot encoding

one_hot_labels = to_categorical(encoded_labels)

<ipython-input-41-b43118fa738d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.fillna("", inplace=True)
<ipython-input-41-b43118fa738d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['combined_text'] = features['Title'] + " " + features['Text']


In [42]:
X_train, X_val, y_train, y_val = train_test_split(
    padded_sequences, one_hot_labels, test_size=0.2, random_state=42
)


In [43]:
train_data.head()

,Title,Political Lean,Score,Id,Subreddit,URL,Num of Comments,Text,Date Created
0,Happy holidays! Socially distanced celebration...,Liberal,24,kh4lsm,feminisms,https://www.nbcnews.com/think/opinion/happy-ho...,0,NaN,1.608506e+09
1,Sandy Hook families settle with Remington mark...,Conservative,90,st7kvu,Libertarian,https://abcnews.go.com/US/sandy-hook-families-...,152,NaN,1.644944e+09
2,What is land claimed versus land owned? : newc...,Conservative,1,3sy3aj,anarchocapitalism,https://www.reddit.com/r/newcountryproject/com...,0,NaN,1.447626e+09
3,What would you say are some skippable/ less im...,Liberal,2,t1po2s,socialism,https://www.reddit.com/r/socialism/comments/t1...,14,Some of the essays and books that were really ...,1.645855e+09
4,2 lawyers arguing against Biden's vaccine mand...,Liberal,730,rybi91,democrats,https://www.businessinsider.com/lawyers-suprem...,64,NaN,1.641573e+09


In [ ]:
# Update the model to have 2 output neurons for one-hot labels
example_CNN_model = Sequential([
    Conv1D(filters=512, kernel_size=3, activation='relu', input_shape=(100, 1)),
    Dropout(0.2),
    GlobalMaxPooling1D(),
    Dense(512, activation='relu'),
    Dense(128, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(2, activation='softmax')  # 2 neurons for 2 classes
])

# Compile the model with categorical cross-entropy
example_CNN_model.compile(
    loss='categorical_crossentropy',  # Use this for one-hot encoded labels
    optimizer='adam',
    metrics=['accuracy']
)

# Train the model
example_CNN_history = example_CNN_model.fit(
    X_train,
    y_train,  # Keep y_train as is (one-hot encoded)
    validation_data=(X_val, y_val),
    epochs=50,
    verbose=1
)


Epoch 1/50
322/322 ━━━━━━━━━━━━━━━━━━━━ 20s 53ms/step - accuracy: 0.5458 - loss: 16.0092 - val_accuracy: 0.4870 - val_loss: 0.7849
Epoch 2/50
322/322 ━━━━━━━━━━━━━━━━━━━━ 11s 34ms/step - accuracy: 0.5543 - loss: 1.6606 - val_accuracy: 0.6332 - val_loss: 0.7405
Epoch 3/50
322/322 ━━━━━━━━━━━━━━━━━━━━ 22s 38ms/step - accuracy: 0.5650 - loss: 0.8970 - val_accuracy: 0.6488 - val_loss: 0.8291
Epoch 4/50
322/322 ━━━━━━━━━━━━━━━━━━━━ 23s 46ms/step - accuracy: 0.5946 - loss: 0.7487 - val_accuracy: 0.6488 - val_loss: 0.7222
Epoch 5/50
322/322 ━━━━━━━━━━━━━━━━━━━━ 18s 37ms/step - accuracy: 0.5802 - loss: 0.7422 - val_accuracy: 0.6468 - val_loss: 0.6661
Epoch 6/50
322/322 ━━━━━━━━━━━━━━━━━━━━ 11s 33ms/step - accuracy: 0.6208 - loss: 0.6851 - val_accuracy: 0.6480 - val_loss: 0.6594
Epoch 7/50
322/322 ━━━━━━━━━━━━━━━━━━━━ 11s 34ms/step - accuracy: 0.6089 - loss: 0.6816 - val_accuracy: 0.6464 - val_loss: 0.6533
Epoch 8/50
322/322 ━━━━━━━━━━━━━━━━━━━━ 21s 36ms/step - accuracy: 0.6176 - loss: 0.6746 -

In [ ]:
print(f'Model Structure and Parameters')
example_CNN_model.summary()

In [ ]:
acc = example_CNN_history.history['accuracy']
val_acc = example_CNN_history.history['val_accuracy']
loss = example_CNN_history.history['loss']
val_loss = example_CNN_history.history['val_loss']

epochs = range(len(acc))

# Plot training and validation accuracy
plt.plot(epochs, acc, 'ro', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy of Liberal and Conversative Classifications')
plt.legend()

plt.figure()

# Plot training and validation loss
plt.plot(epochs, loss, 'ro', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss of Liberal and Conversative Classifications')
plt.legend()

plt.show()